# Etapa 6:  Predicción mediante aprendizaje automático

Space X anuncia en su página web lanzamientos de cohetes Falcon 9 con un coste de 62 millones de dólares; otros proveedores cuestan más de 165 millones de dólares cada uno, gran parte del ahorro se debe a que Space X puede reutilizar la primera etapa. Por lo tanto, si podemos determinar si la primera etapa aterrizará, podemos determinar el coste de un lanzamiento. Esta información puede utilizarse si una empresa alternativa quiere pujar contra Space X por el lanzamiento de un cohete.   En este laboratorio, crearás una línea de aprendizaje automático para predecir si la primera etapa aterrizará dados los datos de los laboratorios anteriores.

## Objetivos

Realizar un análisis exploratorio de los datos y determinar las etiquetas de formación

* Crear una columna para la clase
* Estandarizar los datos
* Dividir en datos de entrenamiento y datos de prueba

\-Encontrar el mejor hiperparámetro para SVM, árboles de clasificación y regresión logística

* Encontrar el método que mejor se comporta utilizando los datos de prueba


***


## Importar librerias y Definir Funciones Auxiliares


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

Esta función sirve para visualizar la matriz de confusión.


In [ ]:
def plot_confusion_matrix(y,y_predict):
    from sklearn.metrics import confusion_matrix

    cm = confusion_matrix(y, y_predict)
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax);
    ax.set_xlabel('Valores predecidos')
    ax.set_ylabel('Valores reales')
    ax.set_title('Confusion Matrix');
    ax.xaxis.set_ticklabels(['no aterrizó', 'aterrizó']); ax.yaxis.set_ticklabels(['no aterrizó', 'aterrizó'])

## Cargar el dataframe


Cargar los datos


In [ ]:
data = pd.read_csv("dataset_part_2.csv")
data.head()

Al conjunto de datos anterior hemos aplicado una transformación one-hot para poder realizar las tareas de machine learning correctamente

In [ ]:
X = pd.read_csv('dataset_one-hot.csv')

X.head(100)

## Tarea  1


Crea un array NumPy a partir de la columna `Class` en `data`, aplicando el método `to_numpy()` y luego
asignarlo a la variable `Y`, asegúrese de que la salida es un array de Numpy (sólo un paréntesis df\['nombre de la columna'\]).

In [ ]:
#Inserta tu código aquí
Y = data.Class.to_numpy()
Y

## Tarea  2


Normalice los datos en `X` usando al función `fit_transform` y luego reasígnelos a la variable `X` utilizando la transformación proporcionada a continuación.


In [ ]:
scaler = preprocessing.StandardScaler()

In [ ]:
#Inserta tu código aquí
X = scaler.fit_transform(X)

Dividimos los datos en datos de entrenamiento y de prueba utilizando la función `train_test_split`.   Los datos de entrenamiento se dividen en datos de validación, un segundo conjunto utilizado para los datos de entrenamiento; luego se entrenan los modelos y se seleccionan los hiperparámetros utilizando la función `GridSearchCV`.


## Tarea  3


Utilice la función `train_test_split` para dividir los datos X e Y en datos de entrenamiento y de prueba como hicimos en la práctica de regresión. Establezca el parámetro test_size en 0,2 y random_state en 2. Los datos de entrenamiento y los datos de prueba deben asignarse a las siguientes etiquetas.


`X_train, X_test, Y_train, Y_test`


In [ ]:
#Inserta tu código aquí
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=11)

## Tarea  4


* Cree un objeto de regresión logística.
* Luego crea un objeto GridSearchCV `logreg_cv` con el objeto de regresión lógistica, los parametros descritos, `scoring='accuracy'` y `cv = 10`.  
* Ajusta el objeto GridSearchCV para encontrar los mejores parámetros del diccionario `parameters`.


In [ ]:
parameters ={'C':[0.01,0.1,1],
             'penalty':['l2'],
             'solver':['lbfgs']}

In [ ]:
#Inserta tu código aquí
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

logreg_cv = GridSearchCV(lr,parameters, scoring='accuracy', cv = 10)
logreg_cv.fit(X_train,Y_train)

Mostramos los mejores parámetros utilizando el atributo de datos `best_params_` y la precisión en los datos de validación utilizando el atributo de datos `best_score_`.


In [ ]:
print("Mejores parámetros ", logreg_cv.best_params_)
print("Precisión de la regresión logística:", logreg_cv.best_score_)

## Tarea  5


Predice `X_test` y guardalo en `y_pred`

In [ ]:
#Inserta tu código aquí
y_pred = logreg_cv.predict(X_test)

Calcule la precisión en los datos de prueba utilizando el método `score`sobre `X_test` e `Y_test`:


In [ ]:
#Inserta tu código aquí
precision = logreg_cv.score(X_test, Y_test)

print("Precisión de la regresión logística mediante el método scorec: ", precision)

Compara `y_pred` con `Y_test` con una matriz de confusión:

In [ ]:
#Inserta tu código aquí
plot_confusion_matrix(Y_test,y_pred)

Al examinar la matriz de confusión, vemos que la regresión logística puede distinguir entre las diferentes clases.  Vemos que el principal problema son los falsos positivos.


## Tarea  6


* Crea un objeto [Support Vector Machine](https://scikit-learn.org/stable/modules/svm.html) y guardalo en svm
* Luego crea un objeto `GridSearchCV` `svm_cv` con el objeto de svm, los parametros descritos, `scoring='accuracy'` y  `cv=10`.  
* Ajusta el objeto para encontrar los mejores parámetros del diccionario `parameters`.


In [ ]:
parameters2 = {'kernel':('linear', 'rbf','poly','rbf', 'sigmoid'),
              'C': np.logspace(-3, 3, 6),
              'gamma':np.logspace(-3, 3, 5)}

In [ ]:
#Inserta tu código aquí
from sklearn import svm

# Crear un objeto Support Vector Machine
svm = svm.SVC()

In [ ]:
#Inserta tu código aquí
svm_cv = GridSearchCV(svm, parameters2, scoring='accuracy', cv=10)

svm_cv.fit(X_train,Y_train)

In [ ]:
print("Mejores parámetros: ", svm_cv.best_params_)
print("Accuracy :", svm_cv.best_score_)

## Tarea  7


Calcule la precisión en los datos de prueba utilizando el método `score`sobre `X_test` e `Y_test`:


In [ ]:
#Inserta tu código aquí
y_pred2 = svm_cv.predict(X_test)

precision2 = svm_cv.score(X_test, Y_test)

print("Precisión de la regresión logística mediante el método scorec: ", precision2)

Podemos representar la matriz de confusión:

In [ ]:
yhat=svm_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

## Tarea  8


* Crea un objeto clasificador de árbol de decisión
* Y luego crea un objeto GridSearchCV tree_cv con el objeto clasificador de árbol de decisión, los parametros descritos, `scoring='accuracy'` y  cv = 10.
* Ajuste el objeto para encontrar los mejores parámetros a partir de los parámetros del diccionario.

In [ ]:
parameters3 = {'criterion': ['gini', 'entropy'],
     'splitter': ['best', 'random'],
     'max_depth': [2*n for n in range(1,10)],
     'max_features': ['auto', 'sqrt'],
     'min_samples_leaf': [1, 2, 4],
     'min_samples_split': [2, 5, 10]}

param_grid3 = {
    'criterion': ['gini', 'entropy'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

In [ ]:
#Inserta tu código aquí
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

# Definir el clasificador de árbol de decisión
tree_classifier = DecisionTreeClassifier()

# Crear el objeto GridSearchCV
tree_cv = GridSearchCV(tree_classifier, param_grid3, scoring='accuracy', cv=10)


In [ ]:
#Inserta tu código aquí
tree_cv.fit(X_train,Y_train)

In [ ]:
print("Mejores parámetros:", tree_cv.best_params_)
print("Accuracy for decision tree classifier:", tree_cv.best_score_)

## Tarea  9


Calcula la precisión de tree_cv en los datos de prueba utilizando el método `score`:


In [ ]:
#Inserta tu código aquí
y_pred3 = tree_cv.predict(X_test)

precision3 = tree_cv.score(X_test, Y_test)

print("Precisión de la regresión logística mediante el método scorec: ", precision3)

Podemos representar la matriz de confusión


In [ ]:
yhat = tree_cv.predict(X_test)
plot_confusion_matrix(Y_test, yhat)

## Tarea  10


* Crear un objeto k vecinos más cercanos
* Y luego crear un objeto `GridSearchCV` `knn_cv` con el objeto k vecinos más cercanos, los parametros descritos, `scoring='accuracy'` y  cv = 10.
* Ajuste el objeto para encontrar los mejores parámetros del diccionario `parameters`.


In [ ]:
parameters4 = {'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2]}

param_grid4 = {
    'n_neighbors': [3, 5, 7],  # Valores que deseas probar para el número de vecinos
    'p': [1, 2]  # Valores que deseas probar para el tipo de métrica de distancia (1 para distancia Manhattan, 2 para distancia Euclidiana)
}


In [ ]:
#Inserta tu código aquí
from sklearn.neighbors import KNeighborsRegressor
knn = KNeighborsRegressor()

# Crear el objeto GridSearchCV
knn_cv = GridSearchCV(knn, parameters4, scoring='accuracy', cv=10)


In [ ]:
#Inserta tu código aquí
knn_cv.fit(X_train, Y_train)

In [1]:
print("Mejores parámetros: ", knn_cv.best_params_)
print("Accuracy for k nearest neighbors:", knn_cv.best_score_)

NameError: ignored

## Tarea  11


Calcula la precisión de knn_cv en los datos de prueba utilizando el método `score`:


In [ ]:
#Inserta tu código aquí
y_pred4 = knn_cv.predict(X_test)

precision4 = knn_cv.score(X_test, Y_test)

print("Precisión de la regresión logística mediante el método scorec: ", precision4)

Podemos representar la matriz de confusión


In [ ]:
yhat = knn_cv.predict(X_test)
plot_confusion_matrix(Y_test,yhat)

## Tarea  12


Encuentra el método que mejor funciona. Para ello crea un dataframe con los los resultados y los algoritmos. Muestra los diferentes resultado en un gráfico de barras.

In [ ]:
algorithms = {'KNN':knn_cv.best_score_,'Decision Tree':tree_cv.best_score_,'Logistic Regression':logreg_cv.best_score_,'SVM':svm_cv.best_score_}
algorithms

In [ ]:
algo_df = pd.DataFrame.from_dict(algorithms, orient='index', columns=['Accuracy'])
#Inserta aquí tu código new
#Resetea el indice del df
algo_df = algo_df.reset_index()
#Renombre las columnas con: {'index': 'Algorithm'}
algo_df = algo_df.rename(columns={'index': 'Algorithm'})

In [ ]:
import plotly.express as px
import plotly.graph_objects as go

fig = px.bar(algo_df, x='Algorithm', y='Accuracy', hover_data=['Algorithm', 'Accuracy'], color='Accuracy', color_continuous_scale='rdylbu')
fig.update_layout(title='Algorithm vs. Accuracy', xaxis_title='Algorithm', yaxis_title='Accuracy' )
fig.show()